## IMPORT

In [ ]:
# Import Tensorflow 2.0
import tensorflow as tf


# other packages
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

## LOAD DATA

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = (train_labels).astype(np.int64)
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

In [ ]:
plt.figure(figsize=(10,10))
random_inds = np.random.choice(60000,36)
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.xlabel(train_labels[image_ind])


## BUILD MODEL

In [ ]:
def build_fc_model():
  fc_model = tf.keras.Sequential([
      # First define a Flatten layer
      tf.keras.layers.Flatten(),

      # '''TODO: Define the activation function for the first fully connected (Dense) layer.'''
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      # tf.keras.layers.Dense(128, activation= '''TODO'''),

      # '''TODO: Define the second Dense layer to output the classification probabilities'''
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
      # [TODO Dense layer to output classification probabilities]

  ])
  return fc_model

model = build_fc_model()

In [ ]:
'''TODO: Experiment with different optimizers and learning rates. How do these affect
    the accuracy of the trained model? Which optimizers and/or learning rates yield
    the best performance?'''
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.build(input_shape=(None, 28, 28, 1))
model.summary()

## TRAIN

In [ ]:
# Define the batch size and the number of epochs to use during training
BATCH_SIZE = 64
EPOCHS = 5

model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS)


In [ ]:
'''TODO: Use the evaluate method to test the model!'''
test_loss, test_acc = model.evaluate(test_images, test_labels) # TODO
# test_loss, test_acc = # TODO

print('Test accuracy:', test_acc)

## DEPLOY

In [ ]:
import os
import io
import IPython.display
from PIL import Image
import base64


In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import numpy as np

def classifier(image):
    #print(type(image))
    imag_gray = image.convert('L')
    newsize = (28, 28)
    imag_gray = imag_gray.resize(newsize)
    img_np = np.asarray(imag_gray.convert('L')).reshape(1, 28, 28, 1)/255.

    results = model.predict(img_np)[0]


    return np.argmax(results)

gr.close_all()
demo = gr.Interface(fn=classifier,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Number")],
                    title="Simple MNIST classifier",
                    description="Classifying mnist images",
                    allow_flagging="never",)

demo.launch(share=True)